In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
from pathlib import Path
from pprint import pprint
# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# mycode
from repo import REPO
from ml_utilities.output_loader.result_loader import SweepResult, JobResult
from ml_utilities.output_loader import create_job_output_loader
from ml_utilities.output_loader.plot import plot_sweep_summary, plot_data_log_values

# 25 Finetune resnet18 on cifar10

Steps: 
- [x] do hypsearch on full dataset
- [] select random samples (prune randomly)
- [] select samples according to prediction depth metric

## A. Experiment config

### Config file

In [3]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: wasp
  gpu_ids: [0,1,2,3]
  runs_per_gpu: 1
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [0]

sweep:
  type: grid
  axes: 
  - parameter: trainer.optimizer.kwargs.lr
    vals: [5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5]
  - parameter: trainer.optimizer.kwargs.weight_decay
    vals: [0.0, 1e-3, 1e-4]
    
start_num: 0

config:
  experiment_data:
    entity: fslgroup
    project_name: tflearning
    experiment_tag: '25'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet18-ft-hypsearch
    experiment_dir: null
    experiment_notes: 
    seed: 0
    gpu_id: 1
  
  model:
    name: timmmodel
    kwargs:
      name: resnet18
      timm_kwargs:
        pretrained: True
  
  trainer:
    training_setup: finetune
    n_epochs: 20
    val_every: 1
    early_stopping_patience: 5
    save_every: 100e3
    batch_size: 256
    optimizer:
      name: AdamW
      kwargs:
        lr: 1e-3
        weight_decay: 0.0
    loss: crossentropy
    num_workers: 6
  
  data:
    name: cifar10
    kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      n_px: 224
"""
cfg = OmegaConf.create(config_yaml)

In [4]:
# run_command = REPO.create_experiment(cfg, override=False)
# print(run_command)

### Sweep result

In [6]:
sweepr = REPO.get_output_loader(cfg)
print(sweepr)

Exp. Tag(start_num): 25(0)
Exp. Name: cifar10-25.0-resnet18-ft-hypsearch
Training setup: finetune
Model name: timmmodel
Dataset name: cifar10
Sweep type: grid
  trainer.optimizer.kwargs.lr: [0.005, 0.001, 0.0005, 0.0001, 5e-05, 1e-05]
  trainer.optimizer.kwargs.weight_decay: [0.0, 0.001, 0.0001]
Seeds: [0]
Num. jobs: 18
Config updated: 2023-03-12 20:57:46
Sweep started:  2023-03-12 20:58:08



In [7]:
failed_jobs = sweepr.get_failed_jobs()
failed_jobs # No failed jobs

([], {})

In [10]:
sweepr.get_summary().sort_values(by='best_val_score', ascending=True)

,best_epoch,best_val_score,trainer.optimizer.kwargs.lr,trainer.optimizer.kwargs.weight_decay,seed
cifar10-25.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0-seed-0--230312_215540,5,0.140873,0.00005,0.0000,0
cifar10-25.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0.0001-seed-0--230312_215628,5,0.140873,0.00005,0.0001,0
cifar10-25.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0.001-seed-0--230312_215543,7,0.141192,0.00005,0.0010,0
cifar10-25.0-resnet18-ft-hypsearch--lr-0.0001-weight_decay-0-seed-0--230312_213710,8,0.141804,0.00010,0.0000,0
cifar10-25.0-resnet18-ft-hypsearch--lr-0.0001-weight_decay-0.0001-seed-0--230312_214109,8,0.141804,0.00010,0.0001,0
cifar10-25.0-resnet18-ft-hypsearch--lr-0.0001-weight_decay-0.001-seed-0--230312_214013,5,0.145786,0.00010,0.0010,0
cifar10-25.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0-seed-0--230312_220028,19,0.148364,0.00001,0.0000,0
cifar10-25.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0.0001-seed-0--230312_221146,19,0.148364,0.00001,0.0001,0
cifar10-25.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0.001-seed-0--230312_221106,19,0.148364,0.00001,0.0010,0
cifar10-25.0-resnet18-ft-hypsearch--lr-0.0005-weight_decay-0.0001-seed-0--230312_213639,8,0.194139,0.00050,0.0001,0


## B. Random subsample sweep

In [14]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: wasp
  gpu_ids: [0,1,2,3]
  runs_per_gpu: 1
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [0]

sweep:
  type: grid
  axes: 
  - parameter: data.sample_selector.kwargs.fraction
    vals: [0.1, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
    
start_num: 0

config:
  experiment_data:
    entity: fslgroup
    project_name: tflearning
    experiment_tag: '25.1'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet18-ft-randomsubsample
    experiment_dir: null
    experiment_notes: 
    seed: 0
    gpu_id: 1
  
  model:
    name: timmmodel
    kwargs:
      name: resnet18
      timm_kwargs:
        pretrained: True
  
  trainer:
    training_setup: finetune
    n_epochs: 15
    val_every: 1
    early_stopping_patience: 15
    save_every: 100e3
    batch_size: 256
    optimizer:
      name: AdamW
      kwargs:
        lr: 0.00005
        weight_decay: 0.0
    loss: crossentropy
    num_workers: 6
  
  data:
    name: cifar10
    kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      n_px: 224
    sample_selector:
      name: random
      kwargs:
        fraction: 0.1

"""
cfg = OmegaConf.create(config_yaml)

In [15]:
run_command = REPO.create_experiment(cfg, override=True)
print(run_command)

python run_sweep.py --config-name cifar10-25.1.0-resnet18-ft-randomsubsample.yaml
